In [17]:
import pandas as pd
data = pd.read_csv('student-mat.csv',delimiter=';')
X = data.iloc[:,:-3]
y = data.iloc[:,-3:]
X.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,yes,no,no,4,3,4,1,1,3,6
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,yes,yes,no,5,3,3,1,1,3,4
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,yes,yes,no,4,3,2,2,3,3,10
3,GP,F,15,U,GT3,T,4,2,health,services,...,yes,yes,yes,3,2,2,1,1,5,2
4,GP,F,16,U,GT3,T,3,3,other,other,...,yes,no,no,4,3,2,1,2,5,4


In [2]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
categorical_cols = X_train.select_dtypes(include=['object', 'category']).columns.tolist()
numerical_cols = X_train.select_dtypes(include=['int64', 'float64']).columns.tolist()
len(categorical_cols)+len(numerical_cols) == X_train.shape[1]

True

In [3]:
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),
        ('cat', OneHotEncoder(handle_unknown="ignore"), categorical_cols),
    ]
)
linear = make_pipeline(preprocessor, LinearRegression())
Dtree = make_pipeline(preprocessor, DecisionTreeRegressor())
linear.fit(X_train, y_train)
y_pred_linear = linear.predict(X_test)
linear_score= mean_squared_error(y_test, y_pred_linear)
Dtree.fit(X_train, y_train)
y_pred_Dtree = Dtree.predict(X_test)
Dtree_score= mean_squared_error(y_test, y_pred_Dtree)
print("linear_score",linear_score)
print("Dtree_score",Dtree_score)
if linear_score < Dtree_score:
    print("线性回归更好")
else: print("决策树回归更好")

linear_score 13.225900858703545
Dtree_score 28.240506329113924
线性回归更好


In [4]:
# from sklearn.model_selection import GridSearchCV
# param_grid = {
#     'decisiontreeregressor__max_depth': [3, 5, 7],
#     'decisiontreeregressor__min_samples_split': [10, 20, 30],
#     'decisiontreeregressor__min_samples_leaf': [5, 10],      # 避免太小
#     'decisiontreeregressor__max_features': ['sqrt', 0.8]     # 随机选部分特征
# }
# Dtree = make_pipeline(preprocessor, DecisionTreeRegressor(random_state=42))
# grid_search = GridSearchCV(Dtree, param_grid, cv=3, scoring='neg_mean_squared_error')
# grid_search.fit(X_train, y_train)
# best_Dtree = grid_search.best_estimator_
# y_pred_best = best_Dtree.predict(X_test)
# best_score = mean_squared_error(y_test, y_pred_best)
# print("最佳参数:", grid_search.best_params_)
# print("最佳交叉验证得分 (neg-MSE):", grid_search.best_score_)
# print("测试集 MSE:", best_score)
#
# 最佳参数: {'decisiontreeregressor__max_depth': 5, 'decisiontreeregressor__max_features': 'sqrt', 'decisiontreeregressor__min_samples_leaf': 10, 'decisiontreeregressor__min_samples_split': 30}
# 最佳交叉验证得分 (neg-MSE): -12.857656406688525
# 测试集 MSE: 14.70922480099072

最佳参数: {'decisiontreeregressor__max_depth': 5, 'decisiontreeregressor__max_features': 'sqrt', 'decisiontreeregressor__min_samples_leaf': 10, 'decisiontreeregressor__min_samples_split': 30}
最佳交叉验证得分 (neg-MSE): -12.857656406688525
测试集 MSE: 14.70922480099072


In [5]:
# from sklearn.ensemble import RandomForestRegressor
#
# rf = make_pipeline(preprocessor, RandomForestRegressor(n_estimators=150, random_state=42))
# rf.fit(X_train, y_train)
# y_pred_rf = rf.predict(X_test)
# rf_score = mean_squared_error(y_test, y_pred_rf)
# print("随机森林 MSE:", rf_score)

# 随机森林 MSE: 11.879752648851381

随机森林 MSE: 11.879752648851381


 多项式回归(degree=2)
 R2分数

In [16]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import r2_score

poly =  PolynomialFeatures(degree=2, include_bias=False)
poly.fit(X_train[numerical_cols], y_train)
lin_reg =  LinearRegression()
lin_reg.fit(X_train[numerical_cols], y_train)
y_pred = lin_reg.predict(X_train[numerical_cols])
print("r2_score:", r2_score(y_train, y_pred))
print("MSE:", mean_squared_error(y_train, y_pred))

r2_score: 0.1853284383188114
MSE: 12.315755913922807
